In [108]:
import gmaps
import pandas as pd
import requests 
import json
import matplotlib.pyplot as plt

In [109]:
%matplotlib notebook

In [110]:
from ipython_config import gkey
print(gkey)

test


In [126]:
data = pd.read_csv('Hospital_General_Information.csv')
hospital_df = pd.DataFrame(data)
hospital_df.columns
hospital_df.columns
# NC = hospital_df.loc[hospital_df['State']=='NC']
NC = hospital_df

In [127]:
with open('beds.json') as jsonfile:
    video_json = json.load(jsonfile)

In [128]:
bed_df = pd.DataFrame(video_json)
bed_df.columns
bed_df=bed_df[['hospital_bed_count','medicare_provider_number' ]]
bed_df = bed_df.rename(columns={'medicare_provider_number':'Facility ID'})
bed_df.head()

,hospital_bed_count,Facility ID
0,99,010007
1,362,010011
2,252,010016
3,106,010018
4,349,010024


In [129]:
del NC['Hospital overall rating footnote']
del NC['Mortality national comparison footnote']
del NC['Safety of care national comparison footnote']
del NC['Readmission national comparison footnote']
del NC['Patient experience national comparison footnote']
del NC['Efficient use of medical imaging national comparison footnote']
del NC['Effectiveness of care national comparison footnote']
del NC['Timeliness of care national comparison footnote']
del NC['Address']
del NC['City']
del NC['State']
del NC['ZIP Code']
del NC['Phone Number']
NC.head(15)
df = pd.merge(NC, bed_df, on = 'Facility ID', how = 'left')

In [131]:
df.head()


,Facility ID,Facility Name,County Name,Hospital Type,Hospital Ownership,Emergency Services,Meets criteria for meaningful use of EHRs,Hospital overall rating,Mortality national comparison,Safety of care national comparison,Readmission national comparison,Patient experience national comparison,Effectiveness of care national comparison,Timeliness of care national comparison,Efficient use of medical imaging national comparison,Location,hospital_bed_count
0,031309,SAGE MEMORIAL HOSPITAL,APACHE,Critical Access Hospitals,Voluntary non-profit - Private,False,NaN,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,NaN,25
1,044021,WOODRIDGE BEHAVIORAL CENTER,CRITTENDEN,Psychiatric,Proprietary,False,NaN,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,POINT (-90.175858 35.152962),NaN
2,100277,DOUGLAS GARDENS HOSPITAL,MIAMI-DADE,Acute Care Hospitals,Voluntary non-profit - Private,False,NaN,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,POINT (-80.191767 25.822972),32
3,104078,SUNCOAST BEHAVIORAL HEALTH CENTER,MANATEE,Psychiatric,Proprietary,False,NaN,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,POINT (-82.612355 27.460155),NaN
4,130063,TREASURE VALLEY HOSPITAL,ADA,Acute Care Hospitals,Proprietary,False,NaN,Not Available,Not Available,Not Available,Not Available,Above the national average,Not Available,Not Available,Not Available,POINT (-116.290332 43.611289),NaN


In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "39.952583,-75.16522",  # philadelphia coords,
    "rankby": "distance",
    "type": "hospital",
    "key": gkey,
}
